# BirdCLEF 2022 : Final Submission
In this notebook we will predict using the trained data.

This is the last one of the 3 notebooks which uses all the finding and the trained models to predict over the test set.

Note : This will be used as a prediction notebook.

So, the conditions should be met are -
1. RT <= 9hrs.
2. Internet Access : Disabled.
3. Usage of public data : Enabled.

# Libraries

In [ ]:
# Libraries for basic data loading and manipulation
import os
import json
import pandas as pd

import torch
import torch.nn as nn
import torchaudio
from torchaudio.transforms import MelSpectrogram
from torchvision.transforms import Resize
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings("ignore")

# Datapaths

In [ ]:
test_dir = "../input/birdclef-2022/test_soundscapes"
test_base_path = "../input/birdclef-2022/test.csv"
class_dict_base_path = "../input/birdclef-2022-saved-weights-and-misc/class_dict.json"
best_acc_mode_base_path = "../input/birdclef-2022-saved-weights-and-misc/birdclef2022-best_accuracy_model.pt"
best_loss_model_base_path = "../input/birdclef-2022-saved-weights-and-misc/birdclef2022-best_loss_model.pt"

## Loading test dataframe

In [ ]:
test_df = pd.read_csv(test_base_path)
test_df.head()

### Loading class labels which will be needed to prepare predictions.

In [ ]:
class_labels = json.load(open(class_dict_base_path, "r"))
num_classes = len(class_labels.keys())
print("Number of class : {}".format(num_classes))

#### Fixing the device

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

In [ ]:
# Convolution shape updating function
def conv_shape(shape, kernel_size, stride, padding):
    H, W = shape[0], shape[1]
    H = ((H - kernel_size + 2*padding) // stride) + 1
    W = ((W - kernel_size + 2*padding) // stride) + 1
    return H, W

# Neural Network :

The neural network class is needed to prepare the skeleton on which we will add the pretrined weights.

In [ ]:
class Conv(nn.Module):
    
    def __init__(self, 
                   in_channels,
                   out_channels,
                   kernel_size,
                   stride=(1,1),
                   padding=(0,0),
                   momentum=0.15):
        super(Conv, self).__init__()
        self.conv_block = nn.Sequential(
            nn.BatchNorm2d(in_channels, momentum = momentum),
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
            nn.ReLU()
        )
        
    def forward(self, x):
        return self.conv_block(x)


class CLEFNetwork(nn.Module):
    
    def __init__(self,
                 num_classes,
                 in_channels = 1,
                 H = 128,
                 W = 128,
                 num_downs = 3):
        super(CLEFNetwork, self).__init__()
        
        self.num_C = num_classes
        self.num_downs = num_downs
        self.in_channels = in_channels
        self.C = 8
        self.H, self.W = self.calc_HW(H, W)
        self.in_conv_block = Conv(self.in_channels, self.C, 7, (2, 2))
        self.conv_block = nn.ModuleList(
                [
                    Conv(self.C * 2**i, self.C * 2**(i+1), 3, (2, 2))
                    for i in range(self.num_downs-1)
                ]
        )
        self.fc_block = nn.Sequential(
                nn.Linear(self.H * self.W * self.C * 2**(self.num_downs - 1), 1024),
                nn.Linear(1024, 1024),
                nn.Linear(1024, self.num_C)
        )
        
    def calc_HW(self, H, W):
        H, W = conv_shape((H, W), 7, 2, 0)
        for num_down in range(self.num_downs - 1):
            H, W = conv_shape((H, W), 3, 2, 0)
        return H, W
        
        
    def forward(self, x):
        x = self.in_conv_block(x)
        for block in self.conv_block:
            x = block(x)
        x = x.view(x.shape[0], -1)
        x = self.fc_block(x)
        return x

### Calling the Netwrok class and updating pretrained weights

In [ ]:
best_model = CLEFNetwork(num_classes)
checkpoint = torch.load(best_loss_model_base_path, map_location = torch.device(device))
best_model.load_state_dict(checkpoint["model"])
print(best_model)

# Prediction Dataset Generation

In [ ]:
class CLEFPredDataset(Dataset):
    
    def __init__(self,
                data_dir,
                meta_df,
                transform = None
                ):
        super(CLEFPredDataset, self).__init__()
        self.data_dir = data_dir
        self.meta_df = meta_df
        self.transform = transform
        
    def __len__(self):
        return len(self.meta_df)
    
    def __getitem__(self, index):
        path = self.meta_df.loc[index, "file_id"]
        path = f"{os.path.join(self.data_dir, path)}.ogg"
        time = self.meta_df.loc[index, "end_time"]
        mono_audio = self.load_audio(path, time)
        mono_audio = mono_audio.unsqueeze(dim=0)
        return mono_audio
        
    def load_audio(self, path, time):
        audio, sample_rate = torchaudio.load(path)
        audio = audio[:, (time-5)*sample_rate: time*sample_rate]
        if self.transform != None:
            for aug in self.transform:
                audio = aug(audio)
        return audio[0,:]

#### adding transformations to equalize the data.

In [ ]:
augm = [
    MelSpectrogram(n_mels = 128),
    Resize((128, 128))
]
augm

In [ ]:
dataset = CLEFPredDataset(test_dir, test_df, transform = augm)

### Preparing a sample submission file just to save the notebook and submit for the competition.
### Note : When the notebook will be submitted the below cells will be working and the genuine output file will be produced.

In [ ]:
test = test_df.copy()
test["target"] = [False for _ in range(len(test))]
imp_features = ["row_id", "target"]
test = test[imp_features]
test.to_csv("submission.csv", index = False)

# Prediction :

In [ ]:
if device == "cuda:0":
    best_model = best_model.cuda()
BATCH_SIZE = 64
test_dl = DataLoader(dataset, batch_size = BATCH_SIZE, shuffle = False)
prediction = []
with torch.no_grad():
    for index, patch in enumerate(test_dl):
        dev_patch = patch.to(device)
        output = best_model(dev_patch)
        if device == "cuda:0":
            output = torch.argmax(output, dim=1).cpu().tolist()
        else:
            output = torch.argmax(output, dim=1).tolist()
        prediction += output
test_df["target"] = prediction
test_df["target"] = test_df["target"].apply(lambda x : class_labels[str(x)])
test_df["target"] = test_df["bird"] == test_df["target"]
imp_features = ["row_id", "target"]
test_df = test_df[imp_features]
test_df.to_csv("submission.csv", index = False)

# Thanks for visiting :)
# Do UPVOTE if you like it :)

### Follow me on [kaggle](https://www.kaggle.com/sagnik1511) , [GitHub](https://www.github.com/sagnik1511) and on [LinkedIn](https://www.linkedin.com/in/sagnik1511)